In [ ]:
import pandas as pd
import tqdm
import openai
import nltk
import os

from pyprojroot import here
from moral_foundations_llms import utils

In [ ]:
api_path = os.path.join(os.environ['HOME'], 'openai/api.txt')

In [ ]:
with open(api_path, 'r') as f:
    openai.api_key = f.read().strip()

In [ ]:
df = pd.read_csv(here('data/aita_processed.csv'))

In [ ]:
failed = []
responses = {}

In [ ]:
for post in tqdm.tqdm(range(1)):
    n_sentences = len(nltk.sent_tokenize(df.iloc[post]['top_comment']))
    system_message = utils.create_system_message(identity="", length=f"{n_sentences} sentences")
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": df['selftext'].iloc[post]}
            ],
            temperature=0.4)
    except openai.error.APIError as e:
        print(f"Post {post}, OpenAI API returned an API Error: {e}")
        failed.append(post)
        continue
    except openai.error.APIConnectionError as e:
        print(f"Post {post}, OpenAI API request failed to connect: {e}")
        failed.append(post)
        continue
    except openai.error.ServiceUnavailableError as e:
        print(f"Post {post}, OpenAI API returned a Service Unavailable Error: {e}")
        failed.append(post)
        continue
        
    answer = response['choices'][0]['message']['content']
    responses[post] = answer
    gpt_dilemma = answer.split('Verdict')[0].replace('Dilemma:', '').strip()
    gpt_label = answer.split('Verdict: ')[1][:3]
    gpt_reason = answer.split('Reasoning:')[-1].strip()
    df.loc[post, 'gpt_dilemma'] = gpt_dilemma
    df.loc[post, 'gpt_label'] = gpt_label
    df.loc[post, 'gpt_reason'] = gpt_reason